#### Generate the spreadsheet template with data validataion

In [ ]:
import sys
sys.path.append('/odm_modules')
import pandas as pd
from common_func import odm_conn
from common_func import odm_dict
df_tid = odm_dict.odm_tid_df_from_csv()
df_col = odm_dict.odm_col_df_from_csv()
df_col = df_col.loc[df_col.NAME !=''] # remove those line which could not be found in systable
df_tid = df_tid.loc[df_tid.NAME !='']
tid_dict = df_tid.set_index('CTID')['CTABNAME'].to_dict()
tid_desc_dict = df_tid.set_index('CTID')['TTID'].to_dict()
df_col['tid_col'] = df_col.CTID + '.' + df_col.CCOLNAME
data_class_dict = df_col.set_index('tid_col').CCLASS.to_dict()

In [ ]:
def get_cols(tid):
    keycol = df_col.loc[(df_col.CTID == tid) 
                        & (df_col.KEYSEQ !=0) 
                        & (~df_col.CCOLNAME.isin(['CMODEL', 'CLANGUAG']))
                       ].CCOLNAME.iloc[0]
    return [keycol, 'T' + keycol[1:]]

In [ ]:
tids = ['RAY', 'RAB', 'R01', 'R03', 'R12', 'R16', 'R23', 'R27', 'RBL']
model = 'WA'
sql_temp = '''
SELECT {cols} FROM ODMPRD.{tbname} 
WHERE CMODEL = '{model}' and clanguag = '' and fdiscont <>'Y' 
'''

with odm_conn.odm_adhoc('prod') as odmprd_adhoc:

    results = {tid: odmprd_adhoc(sql_temp.format(
        cols =', '.join(get_cols(tid)),
        model = model,
        tbname = tid_dict[tid]
    )) for tid in tids}
    dfs = {tid: pd.DataFrame(result).applymap(lambda x: x.strip()) for tid, result in results.items()}
dfs['R03']

In [ ]:
import openpyxl
from openpyxl.utils import quote_sheetname
from openpyxl import Workbook
from openpyxl.worksheet.datavalidation import DataValidation
xlsx_file = 'temp_val6.xlsx'

# Create the workbook and worksheet we'll be working with
#wb = openpyxl.load_workbook(xlsx_file)
wb = Workbook()
dvs = dict()
for tid in tids:
    ws = wb.create_sheet(tid)
    codes = dfs[tid].iloc[:,0]
    descs = dfs[tid].iloc[:, 1]
    for i, value  in enumerate(codes): #Generates 99 "ip" address in the Column A;
        ws['A{}'.format(i+1)].value= value
        ws['B{}'.format(i+1)].value= descs[i]
    dv = DataValidation(type="list",
                        formula1="{}!$A:$A".format(tid)
                        ,allow_blank = True )
    dv.error ='Your entry is not in the list'
    dv.errorTitle = 'Invalid Entry'

    dv.prompt = 'Please select from the list'
    dv.promptTitle = 'List Selection'
    ws.add_data_validation(dv)
    dvs[tid] = dv

In [ ]:
ws = wb['Sheet']
for tid in tids: 
    ws.add_data_validation(dvs[tid])
def value_gen(): 
    for tid in tids: 
        yield tid_dict[tid]
        yield tid_desc_dict[tid]
values = value_gen()        
for row in ws.iter_rows(min_col = 1, max_col = 2, min_row = 1, max_row = len(tids)):
    for cell in row: 
        cell.value = next(values)

for i, tid in enumerate(tids): 
    dvs[tid].add(ws['C{}'.format(i+1)])


In [ ]:
wb.save(xlsx_file)